In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

## Load the CNA data

In [2]:
x_all_08 = pd.read_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr_0.8_threshold_0.6_X_corr_brca.pkl")

In [3]:
x_all_08.isnull().sum().sum()

0

In [4]:
x_all_08['tcga_id'] = list(map(lambda x: x[:12], x_all_08.index))

In [5]:
chr_col = x_all_08.filter(like="chrX", axis=1).columns

In [6]:
x_all_08_no_x = x_all_08.drop(chr_col, axis=1)

In [7]:
x_all_08.head(5)

,chr1_3218610:4370074,chr1_4370075:4460262,chr1_4460263:4603033,chr1_4603034:4605422,chr1_4605423:5904569,chr1_5904570:5904744,chr1_5904745:12230631,chr1_12230632:12233008,chr1_12233009:14251497,chr1_14251498:15700417,...,chrX_138064294:143124651,chrX_143124652:143136341,chrX_143136342:143136764,chrX_143136765:143382601,chrX_143382602:144249600,chrX_144249601:144249991,chrX_144249992:146050893,chrX_146050894:146053429,chrX_146053429:tcga_id,tcga_id
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,-0.000400,-0.0004,-0.0004,-0.0004,-0.00040,-0.0004,-0.000400,-0.0004,-0.000400,TCGA-AA-3693
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,0.006800,0.0068,0.0068,0.0068,0.00680,0.0068,0.006800,0.0068,0.006800,TCGA-4P-AA8J
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,0.770306,0.9363,0.9363,0.9363,0.93630,0.9363,0.668914,0.5941,0.605398,TCGA-24-0979
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.001700,0.0017,0.0017,0.0017,0.00170,0.0017,0.001700,0.0017,0.001700,TCGA-4W-AA9T
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.001200,-0.0012,-0.0012,-0.0012,-0.01069,0.0103,0.010300,0.0103,0.012903,TCGA-5P-A9KC


In [8]:
x_all_08_no_x.head()

,chr1_3218610:4370074,chr1_4370075:4460262,chr1_4460263:4603033,chr1_4603034:4605422,chr1_4605423:5904569,chr1_5904570:5904744,chr1_5904745:12230631,chr1_12230632:12233008,chr1_12233009:14251497,chr1_14251498:15700417,...,chr22_32815347:32820480,chr22_32820481:36533785,chr22_36533786:38779399,chr22_38779400:38790131,chr22_38790132:47741458,chr22_47741459:47753491,chr22_47753492:47990375,chr22_47990376:49049004,chr22_49049004:tcga_id,tcga_id
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,TCGA-AA-3693
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,TCGA-4P-AA8J
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,TCGA-24-0979
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,TCGA-4W-AA9T
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,TCGA-5P-A9KC


In [9]:
translation_table_train = pd.read_pickle('../data/tcga_brca_raw_19036_row_log_norm_train.pkl').iloc[:,:2]
translation_table_test = pd.read_pickle('../data/tcga_brca_raw_19036_row_norm_test.pkl').iloc[:,:2]

In [10]:
translation_table_train.head(5)

,tcga_id,Ciriello_subtype
0,TCGA-A1-A0SK,Basal
1,TCGA-A2-A04P,Basal
2,TCGA-A2-A0CM,Basal
3,TCGA-A2-A0D2,Basal
4,TCGA-A2-A0ST,Basal


In [11]:
x_all_08['tcga_id'].value_counts().head(5)

TCGA-44-2665    7
TCGA-44-2662    7
TCGA-44-2668    7
TCGA-44-2656    7
TCGA-A6-2684    7
Name: tcga_id, dtype: int64

In [12]:
translation_table_test.head(5)

,tcga_id,subtype
0,TCGA-3C-AAAU,LumA
1,TCGA-3C-AALI,Her2
2,TCGA-3C-AALJ,LumB
3,TCGA-3C-AALK,LumA
4,TCGA-4H-AAAK,LumA


### We need to handle the duplicates by selecting the tumor case in that situation

In [13]:
x_all_08.sort_index(inplace=True)

In [14]:
x_all_08[x_all_08.index.str.contains("TCGA-44-4112")]

,chr1_3218610:4370074,chr1_4370075:4460262,chr1_4460263:4603033,chr1_4603034:4605422,chr1_4605423:5904569,chr1_5904570:5904744,chr1_5904745:12230631,chr1_12230632:12233008,chr1_12233009:14251497,chr1_14251498:15700417,...,chrX_138064294:143124651,chrX_143124652:143136341,chrX_143136342:143136764,chrX_143136765:143382601,chrX_143382602:144249600,chrX_144249601:144249991,chrX_144249992:146050893,chrX_146050894:146053429,chrX_146053429:tcga_id,tcga_id
TCGA-44-4112-01A-01D-1877-01.bed,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.006900,...,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.001700,-0.0017,-0.0017,TCGA-44-4112
TCGA-44-4112-01A-01D-A273-01.bed,-0.0278,-0.0278,-0.0278,-0.0278,-0.0278,-0.0278,-0.0278,-0.0278,-0.0278,-0.027800,...,-0.0199,-0.0199,-0.0199,-0.0199,-0.0199,-0.0199,-0.019900,-0.0199,-0.0199,TCGA-44-4112
TCGA-44-4112-01B-06D-A273-01.bed,-0.0908,-0.0908,-0.0908,-0.0908,-0.0908,-0.0908,-0.0908,-0.0908,-0.0908,-0.086012,...,-0.0649,-0.0649,-0.0649,-0.0649,-0.0649,-0.0649,-0.064900,-0.0649,-0.0649,TCGA-44-4112
TCGA-44-4112-10A-01D-1450-01.bed,-0.0047,-0.0047,-0.0047,-0.0047,-0.0047,-0.0047,-0.0047,-0.0047,-0.0047,-0.004700,...,-0.0005,-0.0005,-0.0005,-0.0005,-0.0005,-0.0005,-0.000500,-0.0005,-0.0005,TCGA-44-4112
TCGA-44-4112-10A-01D-1877-01.bed,-0.0085,-0.0085,-0.0085,-0.0085,-0.0085,-0.0085,-0.0085,-0.0085,-0.0085,-0.008500,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.000100,0.0001,0.0001,TCGA-44-4112
TCGA-44-4112-10A-01D-A273-01.bed,-0.0068,-0.0068,-0.0068,-0.0068,-0.0068,-0.0068,-0.0068,-0.0068,-0.0068,-0.006800,...,-0.0005,-0.0005,-0.0005,-0.0005,-0.0005,-0.0005,0.017575,-0.0033,-0.0033,TCGA-44-4112
TCGA-44-4112-11A-01D-1877-01.bed,0.0042,0.0042,0.0042,0.0042,0.0042,0.0042,0.0042,0.0042,0.0042,0.004200,...,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.000200,0.0002,0.0002,TCGA-44-4112


### We can now drop the duplicates as we are sure we will keep the cancer ones

In [15]:
x_all_08.drop_duplicates(subset="tcga_id", keep="first", inplace=True)

In [16]:
# sanity check
x_all_08[x_all_08.index.str.contains("TCGA-44-4112")]

,chr1_3218610:4370074,chr1_4370075:4460262,chr1_4460263:4603033,chr1_4603034:4605422,chr1_4605423:5904569,chr1_5904570:5904744,chr1_5904745:12230631,chr1_12230632:12233008,chr1_12233009:14251497,chr1_14251498:15700417,...,chrX_138064294:143124651,chrX_143124652:143136341,chrX_143136342:143136764,chrX_143136765:143382601,chrX_143382602:144249600,chrX_144249601:144249991,chrX_144249992:146050893,chrX_146050894:146053429,chrX_146053429:tcga_id,tcga_id
TCGA-44-4112-01A-01D-1877-01.bed,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,0.0069,...,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,TCGA-44-4112


In [17]:
x_all_08.shape

(11162, 1674)

In [18]:
x_all_08.to_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr_0.8_threshold_0.6_X_corr_brca_no_dup.pkl")

In [19]:
x_cna_train_08 = pd.merge(x_all_08, translation_table_train, on='tcga_id')
x_cna_test_08 = pd.merge(x_all_08, translation_table_test, on='tcga_id')

In [20]:
x_cna_train_08.head()

,chr1_3218610:4370074,chr1_4370075:4460262,chr1_4460263:4603033,chr1_4603034:4605422,chr1_4605423:5904569,chr1_5904570:5904744,chr1_5904745:12230631,chr1_12230632:12233008,chr1_12233009:14251497,chr1_14251498:15700417,...,chrX_143124652:143136341,chrX_143136342:143136764,chrX_143136765:143382601,chrX_143382602:144249600,chrX_144249601:144249991,chrX_144249992:146050893,chrX_146050894:146053429,chrX_146053429:tcga_id,tcga_id,Ciriello_subtype
0,0.0034,0.0034,0.0034,0.0034,0.0034,0.0034,0.003400,0.0034,0.0034,0.0034,...,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,-0.000957,TCGA-A1-A0SB,Normal
1,-0.3934,-0.3934,-0.3934,-0.3934,-0.3934,-0.3934,-0.393400,-0.3934,-0.3934,-0.3934,...,-0.0019,-0.0019,-0.0019,-0.0019,-0.0019,-0.0019,-0.0019,-0.001900,TCGA-A1-A0SD,LumA
2,-0.0313,-0.0313,-0.0313,-0.0313,-0.0313,-0.0313,-0.031300,-0.0313,-0.0313,-0.0313,...,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.006800,TCGA-A1-A0SE,LumA
3,-0.0562,-0.0562,-0.0562,-0.0562,-0.0562,-0.0562,-0.056200,-0.0562,-0.0562,-0.0562,...,0.0012,0.0012,0.0012,0.0012,0.0012,0.0012,0.0012,0.001200,TCGA-A1-A0SF,LumA
4,-0.3264,-0.3264,-0.3264,-0.3264,-0.3264,-0.3264,-0.316775,-0.3398,-0.3398,-0.3398,...,0.2237,0.2237,0.2237,0.2237,0.2237,0.2237,0.2237,0.223700,TCGA-A1-A0SH,LumA


In [21]:
x_cna_train_08.to_pickle("../data/cna_brca_train_0.8_threshold_0.6_chrX_corr_brca.pkl")
x_cna_test_08.to_pickle("../data/cna_brca_test_0.8_treshold_0.6_chrX_corr_brca.pkl")

## Scale the data and train the model

In [22]:
# For CNA+RNA training
#x_cna_train_08 = pd.read_pickle("../data/hybrids/tcga_brca_cna_rna_meta_train.pkl")
#x_cna_test_08 = pd.read_pickle("../data/hybrids/tcga_brca_cna_rna_meta_test.pkl")

In [23]:
x_cna_train_08.dtypes[x_cna_train_08.dtypes !='float64']

tcga_id             object
Ciriello_subtype    object
dtype: object

In [24]:
x_cna_test_08.dtypes[x_cna_test_08.dtypes !='float64']

tcga_id    object
subtype    object
dtype: object

In [25]:
y_train = x_cna_train_08['Ciriello_subtype']
y_test = x_cna_test_08['subtype']

x_cna_train_08.drop(['tcga_id', 'Ciriello_subtype'], axis=1, inplace=True)
x_cna_test_08.drop(['tcga_id', 'subtype'], axis=1, inplace=True)

In [26]:
y_train.value_counts()

LumA      415
LumB      176
Basal     136
Her2       65
Normal     25
Name: Ciriello_subtype, dtype: int64

In [27]:
y_test.value_counts()

LumA      131
Basal      43
LumB       32
Her2       16
Normal     14
Name: subtype, dtype: int64

In [28]:
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(x_cna_train_08), columns=x_cna_train_08.columns)
X_test = pd.DataFrame(scaler.transform(x_cna_test_08), columns=x_cna_test_08.columns)

In [29]:
values=[0.001, 0.01, 0.1, 1, 10, 100]
i=1
results = pd.DataFrame(columns=["Index", "C", "Accuracy"])
mean_scores = []
all_reports = []
subtypes = ["Basal", "Her2", "LumA", "LumB", "Normal"]


skf = StratifiedKFold(n_splits=5)
for c in values:
    scores = []
    full_report = []

    for train_index, test_index in skf.split(X_train, y_train):
        print("Fold {} of 5".format(i))
        X_cv_train, X_cv_val = X_train.iloc[train_index], X_train.iloc[test_index]
        y_cv_train, y_cv_val = y_train.iloc[train_index], y_train.iloc[test_index]

        clf = LogisticRegression(random_state=0, solver='liblinear', penalty="l1", C=c, multi_class="auto").fit(X_cv_train, y_cv_train)
        
        score = clf.score(X_cv_val, y_cv_val)
        results = results.append({'Fold': i, 'C' : c , 'Score' : score}, ignore_index=True)
        scores.append(score)
        full_report.append(classification_report(y_cv_val, clf.predict(X_cv_val), target_names=subtypes, output_dict=True))
        i+=1
    
    i=1
    mean_scores.append(np.mean(scores))
    all_reports.append(full_report)
    print('Results: {}'.format(scores))
    print('C: {}, Accuracy: {}'.format(c, np.mean(scores)))

mean_scores

Fold 1 of 5


/home/cristovao/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Fold 2 of 5


/home/cristovao/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Fold 3 of 5


...

/home/cristovao/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Fold 5 of 5


/home/cristovao/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Results: [0.7272727272727273, 0.7361963190184049, 0.7055214723926381, 0.7300613496932515, 0.7116564417177914]
C: 1, Accuracy: 0.7221416620189627
Fold 1 of 5
Fold 2 of 5
Fold 3 of 5
Fold 4 of 5
Fold 5 of 5
Results: [0.696969696969697, 0.6319018404907976, 0.6748466257668712, 0.7423312883435583, 0.6748466257668712]
C: 10, Accuracy: 0.6841792154675591
Fold 1 of 5
Fold 2 of 5
Fold 3 of 5
Fold 4 of 5
Fold 5 of 5
Results: [0.6545454545454545, 0.6257668711656442, 0.6319018404907976, 0.6932515337423313, 0.6196319018404908]
C: 100, Accuracy: 0.6450195203569437


[0.1664547313627068,
 0.5079680237962446,
 0.5459155976947387,
 0.7221416620189627,
 0.6841792154675591,
 0.6450195203569437]

In [29]:
from statistics import stdev
stdev([0.7515151515151515, 0.7300613496932515, 0.7300613496932515, 0.7239263803680982, 0.6809815950920245])

0.025878397364798487

In [30]:
clf = LogisticRegression(random_state=0, solver='liblinear', penalty="l1", C=1, multi_class="auto").fit(X_train, y_train)

In [31]:
final_score = clf.score(X_test, y_test)
report = classification_report(y_test, clf.predict(X_test), target_names=subtypes, output_dict=True)
print('Confusion matrix\n', confusion_matrix(y_test, clf.predict(X_test)))
print('Accuracy', final_score)

Confusion matrix
 [[ 39   1   3   0   0]
 [  6   1   4   5   0]
 [  0   1 116  14   0]
 [  2   3  14  13   0]
 [  1   0  11   2   0]]
Accuracy 0.7161016949152542


/home/cristovao/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
report

{'Basal': {'precision': 0.7959183673469388,
  'recall': 0.9069767441860465,
  'f1-score': 0.8478260869565216,
  'support': 43},
 'Her2': {'precision': 0.5,
  'recall': 0.0625,
  'f1-score': 0.1111111111111111,
  'support': 16},
 'LumA': {'precision': 0.7682119205298014,
  'recall': 0.8854961832061069,
  'f1-score': 0.822695035460993,
  'support': 131},
 'LumB': {'precision': 0.4117647058823529,
  'recall': 0.4375,
  'f1-score': 0.42424242424242425,
  'support': 32},
 'Normal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14},
 'micro avg': {'precision': 0.7203389830508474,
  'recall': 0.7203389830508474,
  'f1-score': 0.7203389830508474,
  'support': 236},
 'macro avg': {'precision': 0.4951789987518186,
  'recall': 0.4584945854784307,
  'f1-score': 0.44117493155420995,
  'support': 236},
 'weighted avg': {'precision': 0.6611725507354138,
  'recall': 0.7203389830508474,
  'f1-score': 0.6761996048222705,
  'support': 236}}

### Get the values to fill in the thesis tables

#### Train data

In [34]:
from statistics import stdev

subtypes = ["Basal", "Her2", "LumA", "LumB", "Normal"]
mean_precisions = []
mean_recalls =[]
weights_train=[136,65,415,176,25]

for i in range(0,5):
    dict_aux = all_reports[3][i]
    arr_pre = []
    arr_rec = []
    for sub in subtypes:
        arr_pre.append(dict_aux[sub]['precision'])
        arr_rec.append(dict_aux[sub]['recall'])
    mean_precisions.append(np.average(arr_pre, weights=weights_train))
    mean_recalls.append(np.average(arr_rec, weights=weights_train))

print("PRECISION")
print(mean_precisions)
print('{}+-{}'.format(np.mean(mean_precisions), stdev(mean_precisions)))
print("----------------")
print('RECALL')
print(mean_recalls)
print('{}+-{}'.format(np.mean(mean_recalls), stdev(mean_recalls)))

PRECISION
[0.749707018266184, 0.7017606628377743, 0.7056697986956932, 0.6897205020379639, 0.6770914193243018]
0.7047898802323834+-0.027485407944563712
----------------
RECALL
[0.7515882730081016, 0.7300615880787239, 0.7299812839592521, 0.7237667812943211, 0.6806965734750312]
0.723218899963086+-0.026008757990514445


### Test data

In [43]:
weights_test=[43, 16, 131, 32, 14]
mean_precisions = []
mean_recalls = []


dict_aux = report
arr_pre = []
arr_rec = []
for sub in subtypes:
    arr_pre.append(dict_aux[sub]['precision'])
    arr_rec.append(dict_aux[sub]['recall'])
mean_precisions.append(np.average(arr_pre, weights=weights_test))
mean_recalls.append(np.average(arr_rec, weights=weights_test))
    
print(mean_precisions)
print(mean_recalls)

[0.6611725507354138]
[0.7203389830508474]
